In [ ]:
# leer archivos data/IPH_robos_2015_2020_tecmty.xlsx, data/IPH_robos_2021_2024_tecmty_fechas_corregidas.xlsx, data/IPH_robos_ene-ago-2025_tecmty.xlsx y data/unidades economicas cuadrantes.xlsx y data/poblacion cuadrantes.xlsx
import pandas as pd

# Cargar los archivos
robos_2015_2020 = pd.read_excel("data/IPH_robos_2015_2020_tecmty.xlsx")
robos_2021_2024 = pd.read_excel("data/IPH_robos_2021_2024_tecmty_fechas_corregidas.xlsx")
robos_ene_ago_2025 = pd.read_excel("data/IPH_robos_ene-ago-2025_tecmty.xlsx")
poblacion_cuadrantes = pd.read_excel("data/poblacion cuadrantes.xlsx")

# Robos 2015-2020

In [ ]:
robos_2015_2020.head()

In [ ]:
# ver si en la columna fecha hay datos nulos
robos_2015_2020['FECHA'].isnull().sum()

In [ ]:
# ver que tipos de datos son las columnas
robos_2015_2020.dtypes

# Robos 2021-2024

In [ ]:
robos_2021_2024.head()

In [ ]:
# ver si en la columna fecha hay datos nulos
robos_2021_2024['FECHA'].isnull().sum()

In [ ]:
# ver que tipo de datos son las columnas
robos_2021_2024.dtypes

In [ ]:
# ordenar las fechas de forma ascendente
robos_2021_2024 = robos_2021_2024.sort_values(by='FECHA').reset_index(drop=True)
robos_2021_2024.head()


In [ ]:
# eliminar la columna día
robos_2021_2024 = robos_2021_2024.drop(columns=['dia'])
robos_2021_2024.head()

# Robos 2025

In [ ]:
robos_ene_ago_2025.head()

In [ ]:
# cambiar nombre de columna VOLENCIA a VIOLENCIA
robos_ene_ago_2025 = robos_ene_ago_2025.rename(columns={'VOLENCIA': 'VIOLENCIA'})
robos_ene_ago_2025.head()

In [ ]:
# ver si hay valores nulos en la columna FECHA
robos_ene_ago_2025['FECHA'].isnull().sum()

In [ ]:
# ver que tipos de datos son las columnas
robos_ene_ago_2025.dtypes

# dataframes unidos

In [ ]:
# unir los tres dataframes de robos en uno solo, primero robos_2015_2020, luego robos_2021_2024 y al final robos_ene_ago_2025
robos_tot = pd.concat([robos_2015_2020, robos_2021_2024, robos_ene_ago_2025]).reset_index(drop=True)
robos_tot.head()

In [ ]:
robos_tot.shape

In [ ]:
# ver cuantos cuadrantes unicos hay en el dataframe robos_tot
robos_tot['CUADRANTE'].nunique()

In [ ]:
# crear columna DIA a partir de la columna FECHA
robos_tot['DIA'] = robos_tot['FECHA'].dt.day
robos_tot.head()

In [ ]:
# crear columnas de MES
robos_tot['MES'] = robos_tot['FECHA'].dt.month

# crear columna de estacion de acuerdo al mes y al día, teniendo en cuenta que las estaciones empiezan en los siguientes días:
# Primavera: 21 de marzo y termina el 20 de junio
# Verano: 21 de junio y termina el 22 de septiembre
# Otoño: 23 de septiembre y termina el 20 de diciembre
# Invierno: 21 de diciembre y termina el 20 de marzo
def obtener_estacion(mes, dia):
    if mes == 3 and dia >= 21 or mes in [4, 5] or mes == 6 and dia <= 20:
        return 'Primavera'
    elif mes == 6 and dia >= 21 or mes in [7, 8] or mes == 9 and dia <= 22:
        return 'Verano'
    elif mes == 9 and dia >= 23 or mes in [10, 11] or mes == 12 and dia <= 20:
        return 'Otoño'
    else:
        return 'Invierno'
robos_tot['ESTACION'] = robos_tot.apply(lambda x: obtener_estacion(x['MES'], x['DIA']), axis=1)
robos_tot.head()

In [ ]:
# crear columna de AÑO a partir de la columna FECHA
robos_tot['AÑO'] = robos_tot['FECHA'].dt.year
robos_tot.head()

In [ ]:
robos_tot[robos_tot['FECHA'] == '2016-03-20']

In [ ]:
# exportar robos_tot a un archivo excel llamado robos_tot_final.xlsx
# robos_tot.to_excel('robos_tot_final.xlsx', index=False)

In [ ]:
import matplotlib.pyplot as plt

# --- 1. Asegurar formato de fecha ---
robos_tot['FECHA'] = pd.to_datetime(robos_tot['FECHA'])

# --- 2. Filtrar por mes y tipo de robo ---
mes = 1  # Enero
tipo = "ROBO A CASA HABITACION"  # Cambia esto al tipo que quieras
año = 2025  # Cambia esto al año que quieras

df_filtrado_reporte = robos_tot[(robos_tot['FECHA'].dt.month == mes) & 
                        (robos_tot['TIPO'] == tipo) &
                        (robos_tot['AÑO'] == año)]
# --- 3. Contar robos por día ---
robos_por_dia = df_filtrado_reporte.groupby(df_filtrado_reporte['FECHA'].dt.day).size()

# --- 4. Graficar histograma ---
plt.figure(figsize=(8,5))
plt.hist(robos_por_dia, bins=10)
plt.title(f"Histograma del número de robos diarios - {tipo} (Mes {mes})")
plt.xlabel("Robos por día")
plt.ylabel("Frecuencia")
plt.grid(False)
plt.show()

# Crear tabla de cuadrantes para los robos a negocios

In [ ]:
# crear tabla de 78 cuadrantes, cada fila que sea 1 cuadrante, desde el 1 hasta el 78
cuadrantes_df_negocios = pd.DataFrame({'CUADRANTE': range(1, 79)})
cuadrantes_df_negocios.head()


In [ ]:
# cambiar el nombre de la columna 'cuadrante' a 'CUADRANTE' para hacer merge despues, y también de la columna 'poblacion' a 'POBLACION' en el dataframe poblacion_cuadrantes
poblacion_cuadrantes = poblacion_cuadrantes.rename(columns={'cuadrante': 'CUADRANTE', 'poblacion': 'POBLACION'})
poblacion_cuadrantes.head()

In [ ]:
# crear columna ROBOS A NEGOCIOS MES 1 2015, ROBOS A NEGOCIOS MES 1 2016, ROBOS A NEGOCIOS MES 1 2017, ROBOS A NEGOCIOS MES 1 2018, ROBOS A NEGOCIOS MES 1 2019, ROBOS A NEGOCIOS MES 1 2020, ROBOS A NEGOCIOS MES 1 2021, ROBOS A NEGOCIOS MES 1 2022, ROBOS A NEGOCIOS MES 1 2023, ROBOS A NEGOCIOS MES 1 2024, ROBOS A NEGOCIOS MES 1 2025, PROMEDIO DE ROBOS A NEGOCIOS MES 1, ROBOS A NEGOCIOS MES 2 2015, ROBOS A NEGOCIOS MES 2 2016, ROBOS A NEGOCIOS MES 2 2017, ROBOS A NEGOCIOS MES 2 2018, ROBOS A NEGOCIOS MES 2 2019, ROBOS A NEGOCIOS MES 2 2020, ROBOS A NEGOCIOS MES 2 2021, ROBOS A NEGOCIOS MES 2 2022, ROBOS A NEGOCIOS MES 2 2023, ROBOS A NEGOCIOS MES 2 2024, ROBOS A NEGOCIOS MES 2 2025, PROMEDIO DE ROBOS A NEGOCIOS MES 2..., hasta ROBOS A NEGOCIOS MES 12 2025, PROMEDIO DE ROBOS A NEGOCIOS MES 12
for mes in range(1, 13):
    for año in range(2015, 2025):
        # crear columna robos a negocios por mes y año
        cuadrantes_df_negocios[f'ROBOS A NEGOCIOS MES {mes} {año}'] = cuadrantes_df_negocios['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['MES'] == mes) & (robos_tot['AÑO'] == año) & (robos_tot['TIPO'] == 'ROBO A NEGOCIO')].shape[0])
    # crear columna promedio de robos a negocios por mes, CLACULANDO LA MEDIA DE ROBOS A NEGOCIOS POR MES
    #cuadrantes_df_negocios[f'PROMEDIO DE ROBOS A NEGOCIOS MES {mes}'] = cuadrantes_df_negocios['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['MES'] == mes) & (robos_tot['TIPO'] == 'ROBO A NEGOCIO')].shape[0] / 11)
cuadrantes_df_negocios.head()

In [ ]:
# ver las primeras 13 columnas
cuadrantes_df_negocios.iloc[:, :13].head()

In [ ]:
# ver los nombres de las columnas
cuadrantes_df_negocios.columns

In [ ]:
# ver cuantos cuadrantes hay en el dataframe
cuadrantes_df_negocios['CUADRANTE'].nunique()

In [ ]:
# unir cuadrantes_df_negocios con el dataframe de poblacion_cuadrantes por la columna CUADRANTE, y poner la columna POBLACION despues de la columna CUADRANTE
cuadrantes_df_negocios = pd.merge(cuadrantes_df_negocios, poblacion_cuadrantes, on='CUADRANTE', how='left')
cuadrantes_df_negocios.head()


In [ ]:
# mover la columna POBLACION despues de la columna CUADRANTE
cols = cuadrantes_df_negocios.columns.tolist()
cols.insert(1, cols.pop(cols.index('POBLACION')))
cuadrantes_df_negocios = cuadrantes_df_negocios[cols]
cuadrantes_df_negocios

In [ ]:
# en la columna POBLACION, llenar los valores nulos con 0, y quitar los decimales convirtiendo a entero
cuadrantes_df_negocios['POBLACION'] = cuadrantes_df_negocios['POBLACION'].fillna(0).astype(int)
cuadrantes_df_negocios.head()

In [ ]:
# exportar a excel
cuadrantes_df_negocios.to_excel('cuadrantes_df_negocios_mes_anio.xlsx', index=False)

In [ ]:
# eliminar todas las columnas menos las que dicen PROMEDIO DE ROBOS A NEGOCIOS MES 1, PROMEDIO DE ROBOS A NEGOCIOS MES 2, PROMEDIO DE ROBOS A NEGOCIOS MES 3, PROMEDIO DE ROBOS A NEGOCIOS MES 4, PROMEDIO DE ROBOS A NEGOCIOS MES 5, PROMEDIO DE ROBOS A NEGOCIOS MES 6, PROMEDIO DE ROBOS A NEGOCIOS MES 7, PROMEDIO DE ROBOS A NEGOCIOS MES 8, PROMEDIO DE ROBOS A NEGOCIOS MES 9, PROMEDIO DE ROBOS A NEGOCIOS MES 10, PROMEDIO DE ROBOS A NEGOCIOS MES 11, PROMEDIO DE ROBOS A NEGOCIOS MES 12, y la de poblacion
cols_to_keep = ['CUADRANTE'] + [col for col in cuadrantes_df_negocios.columns if 'PROMEDIO DE ROBOS A NEGOCIOS MES' in col]
cuadrantes_df_negocios = cuadrantes_df_negocios[cols_to_keep]
cuadrantes_df_negocios

In [ ]:
# exportar cuadrantes_df_negocios a un archivo excel llamado cuadrantes_robos_negocios_promedio.xlsx
# cuadrantes_df_negocios.to_excel('cuadrantes_robos_negocios_promedio.xlsx', index=False)

## Crear tabla de cuadrantes para los robos a casa habitación

In [ ]:
# crear tabla de 78 cuadrantes, cada fila que sea 1 cuadrante, desde el 1 hasta el 78
cuadrantes_df_CaHa = pd.DataFrame({'CUADRANTE': range(1, 79)})
cuadrantes_df_CaHa.head()

In [ ]:
poblacion_cuadrantes.head()

In [ ]:
# unir cuadrantes_df_CaHa con poblacion_cuadrantes por la columna CUADRANTE
cuadrantes_df_CaHa = pd.merge(cuadrantes_df_CaHa, poblacion_cuadrantes, on='CUADRANTE', how='left')
cuadrantes_df_CaHa

In [ ]:
# en la columna POBLACION, llenar los valores nulos con 0, y quitar los decimales convirtiendo a entero
cuadrantes_df_CaHa['POBLACION'] = cuadrantes_df_CaHa['POBLACION'].fillna(0).astype(int)
cuadrantes_df_CaHa.head()

In [ ]:
# crear columna ROBOS A CASA HABITACION MES 1 2015, ROBOS A CASA HABITACION MES 1 2016, ROBOS A CASA HABITACION MES 1 2017, ROBOS A CASA HABITACION MES 1 2018, ROBOS A CASA HABITACION MES 1 2019, ROBOS A CASA HABITACION MES 1 2020, ROBOS A CASA HABITACION MES 1 2021, ROBOS A CASA HABITACION MES 1 2022, ROBOS A CASA HABITACION MES 1 2023, ROBOS A CASA HABITACION MES 1 2024, ROBOS A CASA HABITACION MES 1 2025, PROMEDIO DE ROBOS A CASA HABITACION MES 1, ROBOS A CASA HABITACION MES 2 2015, ROBOS A CASA HABITACION MES 2 2016, ROBOS A CASA HABITACION MES 2 2017, ROBOS A CASA HABITACION MES 2 2018, ROBOS A CASA HABITACION MES 2 2019, ROBOS A CASA HABITACION MES 2 2020, ROBOS A CASA HABITACION MES 2 2021, ROBOS A CASA HABITACION MES 2 2022, ROBOS A CASA HABITACION MES 2 2023, ROBOS A CASA HABITACION MES 2 2024, ROBOS A CASA HABITACION MES 2 2025, PROMEDIO DE ROBOS A CASA HABITACION MES 2..., hasta ROBOS A CASA HABITACION MES 12 2025, PROMEDIO DE ROBOS A CASA HABITACION MES 12
for mes in range(1, 13):
    for año in range(2015, 2026):
        # crear columna robos a casa habitacion por mes y año
        cuadrantes_df_CaHa[f'ROBOS A CASA HABITACION MES {mes} {año}'] = cuadrantes_df_CaHa['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['MES'] == mes) & (robos_tot['AÑO'] == año) & (robos_tot['TIPO'] == 'ROBO A CASA HABITACION')].shape[0])
    # crear columna promedio de robos a casa habitacion por mes, CLACULANDO LA MEDIA DE ROBOS A CASA HABITACION POR MES
    cuadrantes_df_CaHa[f'PROMEDIO DE ROBOS A CASA HABITACION MES {mes}'] = cuadrantes_df_CaHa['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['MES'] == mes) & (robos_tot['TIPO'] == 'ROBO A CASA HABITACION')].shape[0] / 11)
# ver las primeras columnas
cuadrantes_df_CaHa.head()

In [ ]:
# eliminar todas las columnas menos las que dicen PROMEDIO DE ROBOS A CASA HABITACION MES 1, PROMEDIO DE ROBOS A CASA HABITACION MES 2, PROMEDIO DE ROBOS A CASA HABITACION MES 3, PROMEDIO DE ROBOS A CASA HABITACION MES 4, PROMEDIO DE ROBOS A CASA HABITACION MES 5, PROMEDIO DE ROBOS A CASA HABITACION MES 6, PROMEDIO DE ROBOS A CASA HABITACION MES 7, PROMEDIO DE ROBOS A CASA HABITACION MES 8, PROMEDIO DE ROBOS A CASA HABITACION MES 9, PROMEDIO DE ROBOS A CASA HABITACION MES 10, PROMEDIO DE ROBOS A CASA HABITACION MES 11, PROMEDIO DE ROBOS A CASA HABITACION MES 12
cols_to_keep1 = ['CUADRANTE'] + ['POBLACION'] + [col for col in cuadrantes_df_CaHa.columns if 'PROMEDIO DE ROBOS A CASA HABITACION' in col]
cuadrantes_df_CaHa = cuadrantes_df_CaHa[cols_to_keep1]
cuadrantes_df_CaHa

In [ ]:
# exportar cuadrantes_df_CaHa a un archivo excel llamado cuadrantes_robos_CaHa_promedio.xlsx
# cuadrantes_df_CaHa.to_excel('cuadrantes_robos_CaHa_promedio.xlsx', index=False)

## Crear tabla de cuadrantes para los robos de vehículos

In [ ]:
# crear tabla de 78 cuadrantes, cada fila que sea 1 cuadrante, desde el 1 hasta el 78
cuadrantes_df_vehiculos = pd.DataFrame({'CUADRANTE': range(1, 79)})
cuadrantes_df_vehiculos.head()

In [ ]:
# unir cuadrantes_df_vehiculos con poblacion_cuadrantes por la columna CUADRANTE
cuadrantes_df_vehiculos = pd.merge(cuadrantes_df_vehiculos, poblacion_cuadrantes, on='CUADRANTE', how='left')
cuadrantes_df_vehiculos.head()

In [ ]:
# en la columna POBLACION, llenar los valores nulos con 0, y quitar los decimales convirtiendo a entero
cuadrantes_df_vehiculos['POBLACION'] = cuadrantes_df_vehiculos['POBLACION'].fillna(0).astype(int)
cuadrantes_df_vehiculos.head()

In [ ]:
# crear columna ROBOS DE VEHICULOS MES 1 2015, ROBOS DE VEHICULOS MES 1 2016, ROBOS DE VEHICULOS MES 1 2017, ROBOS DE VEHICULOS MES 1 2018, ROBOS DE VEHICULOS MES 1 2019, ROBOS DE VEHICULOS MES 1 2020, ROBOS DE VEHICULOS MES 1 2021, ROBOS DE VEHICULOS MES 1 2022, ROBOS DE VEHICULOS MES 1 2023, ROBOS DE VEHICULOS MES 1 2024, ROBOS DE VEHICULOS MES 1 2025, PROMEDIO DE ROBOS DE VEHICULOS MES 1, ROBOS DE VEHICULOS MES 2 2015, ROBOS DE VEHICULOS MES 2 2016, ROBOS DE VEHICULOS MES 2 2017, ROBOS DE VEHICULOS MES 2 2018, ROBOS DE VEHICULOS MES 2 2019, ROBOS DE VEHICULOS MES 2 2020, ROBOS DE VEHICULOS MES 2 2021, ROBOS DE VEHICULOS MES 2 2022, ROBOS DE VEHICULOS MES 2 2023, ROBOS DE VEHICULOS MES 2 2024, ROBOS DE VEHICULOS MES 2 2025, PROMEDIO DE ROBOS DE VEHICULOS MES 2..., hasta ROBOS DE VEHICULOS MES 12 2025, PROMEDIO DE ROBOS DE VEHICULOS MES 12
for mes in range(1, 13):
    for año in range(2015, 2026):
        # crear columna robos a vehiculos por mes y año
        cuadrantes_df_vehiculos[f'ROBOS DE VEHICULOS MES {mes} {año}'] = cuadrantes_df_vehiculos['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['MES'] == mes) & (robos_tot['AÑO'] == año) & (robos_tot['TIPO'] == 'ROBO DE VEHICULO')].shape[0])
    # crear columna promedio de robos a vehiculos por mes, CLACULANDO LA MEDIA DE ROBOS A VEHICULOS POR MES
    cuadrantes_df_vehiculos[f'PROMEDIO DE ROBOS DE VEHICULOS MES {mes}'] = cuadrantes_df_vehiculos['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['MES'] == mes) & (robos_tot['TIPO'] == 'ROBO DE VEHICULO')].shape[0] / 11)
cuadrantes_df_vehiculos.head()

In [ ]:
# eliminar todas las columnas menos las que dicen PROMEDIO DE ROBOS A VEHICULOS MES 1, PROMEDIO DE ROBOS A VEHICULOS MES 2, PROMEDIO DE ROBOS DE VEHICULOS MES 3, PROMEDIO DE ROBOS DE VEHICULOS MES 4, PROMEDIO DE ROBOS DE VEHICULOS MES 5, PROMEDIO DE ROBOS DE VEHICULOS MES 6, PROMEDIO DE ROBOS DE VEHICULOS MES 7, PROMEDIO DE ROBOS DE VEHICULOS MES 8, PROMEDIO DE ROBOS DE VEHICULOS MES 9, PROMEDIO DE ROBOS DE VEHICULOS MES 10, PROMEDIO DE ROBOS DE VEHICULOS MES 11, PROMEDIO DE ROBOS DE VEHICULOS MES 12
cols_to_keep2 = ['CUADRANTE'] + ['POBLACION'] + [col for col in cuadrantes_df_vehiculos.columns if 'PROMEDIO DE ROBOS DE VEHICULOS' in col]
cuadrantes_df_vehiculos = cuadrantes_df_vehiculos[cols_to_keep2]
cuadrantes_df_vehiculos

In [ ]:
# graficar un boxplot de la columna PROMEDIO DE ROBOS DE VEHICULOS MES 1 con la libreria plotly
import plotly.express as px

df_graph_veh = cuadrantes_df_vehiculos
fig = px.box(df_graph_veh, y='PROMEDIO DE ROBOS DE VEHICULOS MES 1', title='Boxplot de PROMEDIO DE ROBOS DE VEHICULOS MES 1')
fig.show()

In [ ]:
# graficar un histograma de la columna PROMEDIO DE ROBOS DE VEHICULOS MES 5 usando plotly

fig = px.histogram(df_graph_veh, x='PROMEDIO DE ROBOS DE VEHICULOS MES 5', title='Histograma de PROMEDIO DE ROBOS DE VEHICULOS MES 5')
fig.show()

In [ ]:
# exportar cuadrantes_df_vehiculos a un archivo excel llamado cuadrantes_robos_vehiculos_promedio.xlsx
# cuadrantes_df_vehiculos.to_excel('cuadrantes_robos_vehiculos_promedio.xlsx', index=False)

In [ ]:
# crear columna total de robos, donde sea la suma de robos cuando el cuadrante sea igual al cuadrante de robos_tot
#cuadrantes_df['TOTAL DE ROBOS'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[robos_tot['CUADRANTE'] == x].shape[0])
#cuadrantes_df_negocios.head()

In [ ]:
# crear columna total de robos con violencia, donde sea la suma de robos con violencia cuando el cuadrante sea igual al cuadrante de robos_tot y la columna VIOLENCIA sea igual a 'SI'
#cuadrantes_df_negocios['ROBOS CON VIOLENCIA'] = cuadrantes_df_negocios['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['VIOLENCIA'] == 'SI')].shape[0])
#cuadrantes_df_negocios.head()

In [ ]:
# crear columna total de robos sin violencia, donde sea la suma de robos sin violencia cuando el cuadrante sea igual al cuadrante de robos_tot y la columna VIOLENCIA sea igual a 'NO'
#cuadrantes_df['ROBOS SIN VIOLENCIA'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['VIOLENCIA'] == 'NO')].shape[0])
#cuadrantes_df.head()

In [ ]:
# crear columna Robos a negocios, donde sea la suma de robos a negocios cuando el cuadrante sea igual al cuadrante de robos_tot y la columna TIPO sea igual a 'ROBO A NEGOCIO'
#cuadrantes_df['ROBOS A NEGOCIOS'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['TIPO'] == 'ROBO A NEGOCIO')].shape[0])

# crear columna robos a negocios con violencia, donde sea la suma de robos a negocios con violencia cuando el cuadrante sea igual al cuadrante de robos_tot, la columna TIPO sea igual a 'ROBO A NEGOCIO' y la columna VIOLENCIA sea igual a 'SI'
#cuadrantes_df['ROBOS A NEGOCIOS CON VIOLENCIA'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['TIPO'] == 'ROBO A NEGOCIO') & (robos_tot['VIOLENCIA'] == 'SI')].shape[0])

# crear columna robos a negocios sin violencia, donde sea la suma de robos a negocios sin violencia cuando el cuadrante sea igual al cuadrante de robos_tot, la columna TIPO sea igual a 'ROBO A NEGOCIO' y la columna VIOLENCIA sea igual a 'NO'
#cuadrantes_df['ROBOS A NEGOCIOS SIN VIOLENCIA'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['TIPO'] == 'ROBO A NEGOCIO') & (robos_tot['VIOLENCIA'] == 'NO')].shape[0])
#cuadrantes_df.head()

In [ ]:
# crear columna Robos a casa habitacion, donde sea la suma de robos A CASA HABITACION cuando el cuadrante sea igual al cuadrante de robos_tot y la columna TIPO sea igual a 'ROBO A CASA HABITACION'
#cuadrantes_df['ROBOS A CASA HABITACION'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['TIPO'] == 'ROBO A CASA HABITACION')].shape[0])

# crear columna robos a casa habitacion con violencia, donde sea la suma de robos a casa habitacion con violencia cuando el cuadrante sea igual al cuadrante de robos_tot, la columna TIPO sea igual a 'ROBO A CASA HABITACION' y la columna VIOLENCIA sea igual a 'SI'
#cuadrantes_df['ROBOS A CASA HABITACION CON VIOLENCIA'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['TIPO'] == 'ROBO A CASA HABITACION') & (robos_tot['VIOLENCIA'] == 'SI')].shape[0])

# crear columna robos a casa habitacion sin violencia, donde sea la suma de robos a casa habitacion sin violencia cuando el cuadrante sea igual al cuadrante de robos_tot, la columna TIPO sea igual a 'ROBO A CASA HABITACION' y la columna VIOLENCIA sea igual a 'NO'
#cuadrantes_df['ROBOS A CASA HABITACION SIN VIOLENCIA'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['TIPO'] == 'ROBO A CASA HABITACION') & (robos_tot['VIOLENCIA'] == 'NO')].shape[0])
#cuadrantes_df.head()

In [ ]:
# crear columna Robos DE VEHICULOS, donde sea la suma de robos DE VEHICULOS cuando el cuadrante sea igual al cuadrante de robos_tot y la columna TIPO sea igual a 'ROBO DE VEHICULO'
#cuadrantes_df['ROBOS DE VEHICULOS'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['TIPO'] == 'ROBO DE VEHICULO')].shape[0])

# crear columna robos de vehiculos con violencia, donde sea la suma de robos de vehiculos con violencia cuando el cuadrante sea igual al cuadrante de robos_tot, la columna TIPO sea igual a 'ROBO DE VEHICULO' y la columna VIOLENCIA sea igual a 'SI'
#cuadrantes_df['ROBOS DE VEHICULOS CON VIOLENCIA'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['TIPO'] == 'ROBO DE VEHICULO') & (robos_tot['VIOLENCIA'] == 'SI')].shape[0])

# crear columna robos de vehiculos sin violencia, donde sea la suma de robos de vehiculos sin violencia cuando el cuadrante sea igual al cuadrante de robos_tot, la columna TIPO sea igual a 'ROBO DE VEHICULO' y la columna VIOLENCIA sea igual a 'NO'
#cuadrantes_df['ROBOS DE VEHICULOS SIN VIOLENCIA'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['TIPO'] == 'ROBO DE VEHICULO') & (robos_tot['VIOLENCIA'] == 'NO')].shape[0])
#cuadrantes_df.head()

In [ ]:
'''''
crear columnas de promedio de robos, promedio de robos con y sin violencia, promedio de robos a negocios, promedio de robos a negocios con y sin violencia, promedio de robos a casa habitacion, promedio de robos a casa habitacion con y sin violencia, promedio de robos de vehiculos, promedio de robos de vehiculos con y sin violencia para todos los meses para todos los 10 años
#for mes in range(1, 13):
    # crear columna promedio de robos por mes, CLACULANDO LA MEDIA DE ROBOS POR MES
    cuadrantes_df[f'PROMEDIO DE ROBOS MES {mes}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['MES'] == mes)].shape[0])
    # crear columna promedio de robos con violencia por mes
    cuadrantes_df[f'PROMEDIO DE ROBOS CON VIOLENCIA MES {mes}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['MES'] == mes) & (robos_tot['VIOLENCIA'] == 'SI')].shape[0])
    # crear columna promedio de robos sin violencia por mes
    cuadrantes_df[f'PROMEDIO DE ROBOS SIN VIOLENCIA MES {mes}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['MES'] == mes) & (robos_tot['VIOLENCIA'] == 'NO')].shape[0])
    # crear columna promedio de robos a negocios por mes
    cuadrantes_df[f'PROMEDIO DE ROBOS A NEGOCIOS MES {mes}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['MES'] == mes) & (robos_tot['TIPO'] == 'ROBO A NEGOCIO')].shape[0])
    # crear columna promedio de robos a negocios con violencia por mes
    cuadrantes_df[f'PROMEDIO DE ROBOS A NEGOCIOS CON VIOLENCIA MES {mes}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['MES'] == mes) & (robos_tot['TIPO'] == 'ROBO A NEGOCIO') & (robos_tot['VIOLENCIA'] == 'SI')].shape[0])
    # crear columna promedio de robos a negocios sin violencia por mes
    cuadrantes_df[f'PROMEDIO DE ROBOS A NEGOCIOS SIN VIOLENCIA MES {mes}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['MES'] == mes) & (robos_tot['TIPO'] == 'ROBO A NEGOCIO') & (robos_tot['VIOLENCIA'] == 'NO')].shape[0])
    # crear columna promedio de robos a casa habitacion por mes
    cuadrantes_df[f'PROMEDIO DE ROBOS A CASA HABITACION MES {mes}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['MES'] == mes) & (robos_tot['TIPO'] == 'ROBO A CASA HABITACION')].shape[0])
    # crear columna promedio de robos a casa habitacion con violencia por mes
    cuadrantes_df[f'PROMEDIO DE ROBOS A CASA HABITACION CON VIOLENCIA MES {mes}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['MES'] == mes) & (robos_tot['TIPO'] == 'ROBO A CASA HABITACION') & (robos_tot['VIOLENCIA'] == 'SI')].shape[0])
    # crear columna promedio de robos a casa habitacion sin violencia por mes
    cuadrantes_df[f'PROMEDIO DE ROBOS A CASA HABITACION SIN VIOLENCIA MES {mes}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['MES'] == mes) & (robos_tot['TIPO'] == 'ROBO A CASA HABITACION') & (robos_tot['VIOLENCIA'] == 'NO')].shape[0])
    # crear columna promedio de robos de vehiculos por mes
    cuadrantes_df[f'PROMEDIO DE ROBOS DE VEHICULOS MES {mes}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['MES'] == mes) & (robos_tot['TIPO'] == 'ROBO DE VEHICULO')].shape[0])
    # crear columna promedio de robos de vehiculos con violencia por mes
    cuadrantes_df[f'PROMEDIO DE ROBOS DE VEHICULOS CON VIOLENCIA MES {mes}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['MES'] == mes) & (robos_tot['TIPO'] == 'ROBO DE VEHICULO') & (robos_tot['VIOLENCIA'] == 'SI')].shape[0])
    # crear columna promedio de robos de vehiculos sin violencia por mes
    cuadrantes_df[f'PROMEDIO DE ROBOS DE VEHICULOS SIN VIOLENCIA MES {mes}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['MES'] == mes) & (robos_tot['TIPO'] == 'ROBO DE VEHICULO') & (robos_tot['VIOLENCIA'] == 'NO')].shape[0])
cuadrantes_df.head()
'''''

In [ ]:
'''''''''
cuadrantes_df.head()
'''''''''

In [ ]:
'''
# crear columnas de total de robos, total de robos con y sin violencia, robos a negocios, robos a negocios con y sin violencia, robos a a casa habitacion, robos a casa habitacion con y sin violencia, robos de vehiculos, robos de vehiculos con y sin violencia para todas las estaciones
# ten en cuenta que las estaciones empiezan en los siguientes días: primavera: 21 de marzo, Verano: 21 de junio, Otoño: 23 de septiembre, Invierno: 21 de diciembre
estaciones = ['Primavera', 'Verano', 'Otoño', 'Invierno']
for estacion in estaciones:
    # crear columna total de robos por estacion
    cuadrantes_df[f'TOTAL DE ROBOS {estacion.upper()}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['ESTACION'] == estacion)].shape[0])
    
    # crear columna total de robos con violencia por estacion
    cuadrantes_df[f'ROBOS CON VIOLENCIA {estacion.upper()}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['ESTACION'] == estacion) & (robos_tot['VIOLENCIA'] == 'SI')].shape[0])
    
    # crear columna total de robos sin violencia por estacion
    cuadrantes_df[f'ROBOS SIN VIOLENCIA {estacion.upper()}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['ESTACION'] == estacion) & (robos_tot['VIOLENCIA'] == 'NO')].shape[0])
    
    # crear columna robos a negocios por estacion
    cuadrantes_df[f'ROBOS A NEGOCIOS {estacion.upper()}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['ESTACION'] == estacion) & (robos_tot['TIPO'] == 'ROBO A NEGOCIO')].shape[0])

    # crear columna robos a negocios con violencia por estacion
    cuadrantes_df[f'ROBOS A NEGOCIOS CON VIOLENCIA {estacion.upper()}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['ESTACION'] == estacion) & (robos_tot['TIPO'] == 'ROBO A NEGOCIO') & (robos_tot['VIOLENCIA'] == 'SI')].shape[0])

    # crear columna robos a negocios sin violencia por estacion
    cuadrantes_df[f'ROBOS A NEGOCIOS SIN VIOLENCIA {estacion.upper()}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['ESTACION'] == estacion) & (robos_tot['TIPO'] == 'ROBO A NEGOCIO') & (robos_tot['VIOLENCIA'] == 'NO')].shape[0])

    # crear columna robos a casa habitacion por estacion
    cuadrantes_df[f'ROBOS A CASA HABITACION {estacion.upper()}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['ESTACION'] == estacion) & (robos_tot['TIPO'] == 'ROBO A CASA HABITACION')].shape[0])

    # crear columna robos a casa habitacion con violencia por estacion
    cuadrantes_df[f'ROBOS A CASA HABITACION CON VIOLENCIA {estacion.upper()}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['ESTACION'] == estacion) & (robos_tot['TIPO'] == 'ROBO A CASA HABITACION') & (robos_tot['VIOLENCIA'] == 'SI')].shape[0])

    # crear columna robos a casa habitacion sin violencia por estacion
    cuadrantes_df[f'ROBOS A CASA HABITACION SIN VIOLENCIA {estacion.upper()}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['ESTACION'] == estacion) & (robos_tot['TIPO'] == 'ROBO A CASA HABITACION') & (robos_tot['VIOLENCIA'] == 'NO')].shape[0])

    # crear columna robos de vehiculos por estacion
    cuadrantes_df[f'ROBOS DE VEHICULOS {estacion.upper()}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['ESTACION'] == estacion) & (robos_tot['TIPO'] == 'ROBO DE VEHICULO')].shape[0])

    # crear columna robos de vehiculos con violencia por estacion
    cuadrantes_df[f'ROBOS DE VEHICULOS CON VIOLENCIA {estacion.upper()}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['ESTACION'] == estacion) & (robos_tot['TIPO'] == 'ROBO DE VEHICULO') & (robos_tot['VIOLENCIA'] == 'SI')].shape[0])

    # crear columna robos de vehiculos sin violencia por estacion
    cuadrantes_df[f'ROBOS DE VEHICULOS SIN VIOLENCIA {estacion.upper()}'] = cuadrantes_df['CUADRANTE'].apply(lambda x: robos_tot[(robos_tot['CUADRANTE'] == x) & (robos_tot['ESTACION'] == estacion) & (robos_tot['TIPO'] == 'ROBO DE VEHICULO') & (robos_tot['VIOLENCIA'] == 'NO')].shape[0])

cuadrantes_df.head()
'''

In [ ]:
'''
# separar cuadrantes_df en tres dataframes: uno que tenga las columnas de total de robos, total de robos con y sin violencia, y que tenga todo los demas solo para el tipo de robo de negocio; uno que tenga las columnas de total de robos, total de robos con y sin violencia, y que tenga todo los demas solo para el tipo de robo a casa habitacion; y uno que tenga las columnas de total de robos, total de robos con y sin violencia, y que tenga todo los demas solo para el tipo de robo de vehiculo
cuadrantes_negocios_df = cuadrantes_df[['CUADRANTE', 'POBLACION', 'TOTAL DE ROBOS', 'ROBOS CON VIOLENCIA', 'ROBOS SIN VIOLENCIA'] + 
                                 [col for col in cuadrantes_df.columns if 'NEGOCIOS' in col]]
cuadrantes_negocios_df.head()
'''

In [ ]:
# exportar cuadrantes_negocios_df a un archivo excel llamado cuadrantes_negocios.xlsx
# cuadrantes_negocios_df.to_excel('cuadrantes_negocios.xlsx', index=False)

In [ ]:
'''
cuadrantes_casahabitacion_df = cuadrantes_df[['CUADRANTE', 'POBLACION', 'TOTAL DE ROBOS', 'ROBOS CON VIOLENCIA', 'ROBOS SIN VIOLENCIA'] + 
                                         [col for col in cuadrantes_df.columns if 'CASA HABITACION' in col]]
cuadrantes_casahabitacion_df.head()
'''

In [ ]:
# exportar cuadrantes_casahabitacion_df a un archivo excel llamado cuadrantes_casahabitacion.xlsx
# cuadrantes_casahabitacion_df.to_excel('cuadrantes_casahabitacion.xlsx', index=False)

In [ ]:
'''cuadrantes_vehiculos_df = cuadrantes_df[['CUADRANTE', 'POBLACION', 'TOTAL DE ROBOS', 'ROBOS CON VIOLENCIA', 'ROBOS SIN VIOLENCIA'] + 
                                   [col for col in cuadrantes_df.columns if 'VEHICULOS' in col]]
cuadrantes_vehiculos_df.head()'''

In [ ]:
'''# exportar cuadrantes_vehiculos_df a un archivo excel llamado cuadrantes_vehiculos.xlsx
cuadrantes_vehiculos_df.to_excel('cuadrantes_vehiculos.xlsx', index=False)
'''''